# Day 1

## Part 1

In [1]:
with open("./inputs/input-1.txt", "r") as file:
    numbers = [int(line) for line in file]
    increased_values = list(filter(lambda x: x[1] > x[0], zip(numbers, numbers[1:])))

result = len(increased_values)
print(result)
assert result == 1292
        

1292


## Part 2

In [2]:
with open("./inputs/input-1.txt", "r") as file:
    numbers = [int(line) for line in file]
    triplets = list(zip(numbers, numbers[1:], numbers[2:]))
    sums = list(map(lambda t: sum(t), triplets))
    increased_values = list(filter(lambda x: x[1] > x[0], zip(sums, sums[1:])))
    
result = len(increased_values)
print(result)
assert result == 1262

1262


# Day 2

## Part 1

In [3]:
def get_instruction_value(instruction, list):
    return sum(map(lambda entry: int(entry[1]), filter(lambda entry: entry[0] == instruction, list)))

with open("./inputs/input-2.txt", "r") as file:
    instructions = [tuple(line.split()) for line in file]

forward, down, up = map(lambda dir: get_instruction_value(dir, instructions), ['forward', 'down', 'up'])

result = forward * (down - up)

print(result)
assert result == 1882980

1882980


## Part 2

Trying something new, using a generator function

In [4]:
def movement_generator():
    aim = 0
    for line in open("./inputs/input-2.txt", "r"):
        instruction, value = tuple(line.split())
        value = int(value)
        if instruction == 'forward':
            yield (aim, value)
        else:
            aim = aim + value if instruction == 'down' else aim - value

horizontal = depth = 0

for aim, value in movement_generator():
    horizontal += value
    depth += aim * value

result = horizontal * depth

print(result)
assert result == 1971232560

1971232560


# Day 3

## Part 1

In [5]:
input_file = "./inputs/input-3.txt"

numbers = []
for line in open(input_file, "r"):
    numbers.append(int(line, base=2))

# Determine the given binary length
bit_size = len(open(input_file, "r").readline().rstrip())

# Index is zero-based from the LSB
def get_most_common_bit(index, numbers):
    cnt = 0
    for number in numbers:
        if (number >> index) & 1 == 1:
            cnt += 1
        # I could exit the loop earlier... However ¯\_(ツ)_/¯
    return int(cnt >= (len(numbers) / 2))

def most_common_bits():
    for idx in range(bit_size - 1, -1, -1):
        yield (idx, get_most_common_bit(idx, numbers))

gamma = epsilon = 0

for idx, val in most_common_bits():
    gamma |= val << idx

# Flip all bits in gamma, don't question the logic
epsilon = gamma ^ (2 ** bit_size - 1)
result = epsilon * gamma

print(result)
assert result == 3374136

3374136


## Part 2

In [6]:
# Take read values from part 1
co2_numbers = oxygen_numbers = numbers

def filter_common_bit(index, numbers, lcb=False):
    cb = get_most_common_bit(index, numbers)
    cb = cb if not lcb else cb ^ 0b1
    return list(filter(lambda x: (x >> index) & 1 == cb, numbers))

oxygen_rating = 0
co2_rating = 0

# Probably this would be a case for reudce
for i in range(bit_size - 1, -1, -1):
    oxygen_numbers = filter_common_bit(i, oxygen_numbers)
    co2_numbers = filter_common_bit(i, co2_numbers, True)
    if len(oxygen_numbers) == 1:
        oxygen_rating = oxygen_numbers[0]
    if len(co2_numbers) == 1:
        co2_rating = co2_numbers[0]

result = oxygen_rating * co2_rating
print(result)

assert result == 4432698

4432698


# Day 4

## Part 1

In [2]:
# TIL that it is possible to use type annotations in python
from typing import List, Set, Iterable

input_file = "./inputs/input-4.txt"

bingo_fields = []
with open(input_file, "r") as f:
    # First line contains the drawn bingo numbers
    bingo_numbers = [int(x) for x in f.readline().strip().split(",")]

    # Afterwards read every block as a matrix
    bingo_block = []
    for line in f.readlines()[1:]:
        line = line.strip()
        if line:
            bingo_block.append([int(x) for x in line.split()])
        else:
            bingo_fields.append(bingo_block)
            bingo_block = []


# Verify that the win condition for one specific bingo field is met
# Does only verify vertical and horizontal completeness by using 
# set intersection.
def win_condition_met(bingo_field: Iterable[Iterable[int]], drawn_numbers: Iterable[int]):
    drawn_numbers = set(drawn_numbers)
    for row in bingo_field:
        if len(drawn_numbers.intersection(row)) == 5:
            return True
    for i in range(5):
        col = [row[i] for row in bingo_field]
        if len(drawn_numbers.intersection(col)) == 5:
            return True
    return False


# Calculate the score of one specific bingo field with the drawn numbers
def calculate_score(bingo_field: Iterable[Iterable[int]], drawn_numbers: Iterable[int]):
    flatten_field = set(sum(bingo_field, []))                   # Unpack matrix
    unmarked_numbers = flatten_field.difference(drawn_numbers)
    winning_number = drawn_numbers[-1]                          # Last drawn number
    return sum(unmarked_numbers) * winning_number



drawn_numbers = []
for number in bingo_numbers:
    drawn_numbers.append(number)
    # Find all fields where the win condition is met
    winning_fields = [x for x in bingo_fields if win_condition_met(x, drawn_numbers)]
    
    # If there is a winner available
    if len(winning_fields) >= 1:
        winning_field = winning_fields[0]
        winning_number = number
        break

result = calculate_score(winning_field, drawn_numbers)

print(result)
assert result == 45031


45031


## Part 2

In [3]:
# We're going backwards, so every number is drawn
drawn_numbers = bingo_numbers

# Remove all fields, that would not have won anyway
winning_fields = [x for x in bingo_fields if win_condition_met(x, drawn_numbers)]

# Loop backwars over the drawn bingo numbers, skip 1 position
# Goal is to find the first bingo field that won't met the win condition anymore when
# the drawn numbers are removed in reverse
for i in range(len(bingo_numbers) - 2, 0, -1):
    drawn_numbers = bingo_numbers[:i]
    losing_bingo_fields = [x for x in winning_fields if not win_condition_met(x, drawn_numbers)]
    # If we found a loser 
    if len(losing_bingo_fields) >= 1:
        last_winning_field = losing_bingo_fields[0]
        # We removed the number that was called for the win of this field, therefore + 1
        winning_drawn_numbers = bingo_numbers[:i + 1]
        break
    

result = calculate_score(last_winning_field, winning_drawn_numbers)
print(result)

assert result == 2568

2568


# Day 5

## Part 1

In [3]:
from typing import List, Tuple, Set, Dict
import re

input_file = "./inputs/input-5.txt"

with open(input_file, "r") as f:
    # Just find all the digits and store them in a tuple
    coordinates = [tuple([int (x) for x in re.findall('\d+', line)]) for line in f.readlines()]

# Filter out diagonals
straight_lines = [c for c in coordinates if c[0] == c[2] or c[1] == c[3]]

# This will be the "grid" in which the counting happens
grid = {}

# Take a input of one tuple entry and return all the points (as 2d-tuples) which 
# form a line on the grid
# will only work with straight lines
def expand_points(points: Tuple[int, int, int, int]) -> Set[Tuple[int, int]]:
    x1, y1, x2, y2 = points
    return {(x, y) for x in range(min(x1, x2), max(x2, x1) + 1) for y in range(min(y1, y2), max(y1, y2) + 1)}

# Take a grid as input and count all the entries that are ticked more than once
def count_overlapping_points(grid: Dict[Tuple[int, int], int]) -> int:
    return len([g for g in grid.values() if g > 1])

for c in straight_lines:
    for p in expand_points(c):
        grid[p] = grid.get(p, 0) + 1

result = count_overlapping_points(grid)
print(result)
assert result == 7436

7436


## Part 2

In [4]:
from typing import Tuple, Set
# Reset the grid
grid = {}

# For part 2 it is required to adjust the expand_points function
# so that it will also work with diagonals
def expand_points(points: Tuple[int, int, int, int]) -> Set[Tuple[int, int]]:
    x1, y1, x2, y2 = points

    # This will now only be the case for straight lines
    if x1 == x2 or y1 == y2:
        return {(x, y) for x in range(min(x1, x2), max(x2, x1) + 1) for y in range(min(y1, y2), max(y1, y2) + 1)}
    # Otherwise it is diagonal at exactly 45 degrees
    else:
        # Well, this is some ugly piece of code. But since I noticed, that I don't have much fun anymore
        # it is the solution for now -- will refactor it maybe later on
        if x1 < x2:
            if y1 < y2:
                # Case (0,0 -> 8,8)
                return {(x, y) for (x, y) in zip(range(x1, x2 + 1), range(y1, y2 + 1))}
            else:
                # Case (0,8 -> 8,0)
                return {(x, y) for (x, y) in zip(range(x1, x2 + 1), range(y1, y2 - 1, -1))}
        else: 
            if y1 < y2:
                # Case (8,0) -> (0,8)
                return {(x, y) for (x, y) in zip(range(x1, x2 - 1, -1), range(y1, y2 + 1))}
            else:
                # Case (8,8) -> (0,0)
                return {(x, y) for (x, y) in zip(range(x1, x2 - 1, -1), range(y1, y2 - 1, -1))}

# Now iterate over all coordinates
for c in coordinates:
    for p in expand_points(c):
        grid[p] = grid.get(p, 0) + 1

result = count_overlapping_points(grid)
print(result)
assert result == 21104

21104


# Day 6

## Part 1

In [38]:
from typing import Dict

input_file = "./inputs/input-6.txt"

input_fish_state = []
with open(input_file, "r") as f:
    input_fish_state = [int(x) for x in f.readline().strip().split(",")]

# Define a initial dictionary in which the count of fish states will be stored
fish_state = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0
}

# Fill with input values
for i in input_fish_state:
    fish_state[i] += 1

# Simulates a number of days over the input states
def simulate_days(i_state: Dict[int, int], days: int) -> Dict[int, int]:
    # Copy to not modify the reference
    for day in range(0, days):
        # Retreive the amount of 0 state
        how_much_is_the_fish = i_state[0]
        # Iterate over each state and move the amount of values 
        # from the ascending state to the actual state
        for state in range(0, 8):
            i_state[state] = i_state[state + 1]
        
        # Reset fishes and new fishes
        i_state[8] = how_much_is_the_fish
        i_state[6] += how_much_is_the_fish
    return i_state

# Sum the number of fishes after 80 days        
result = sum(simulate_days(fish_state, 80).values())
print(result)

assert result == 362666


362666


## Part 2

In [39]:
# Sum the number of fishes after 256 days
# Already calculated the first 80 days, therefore we can use it
result = sum(simulate_days(fish_state, 256 - 80).values())
print(result)

assert result == 1640526601595

1640526601595


# Day 7

## Part 1

## Part 2

# Day 8

## Part 1

## Part 2

# Day 9

## Part 1

## Part 2

# Day 10

## Part 1

## Part 2

# Day 11

## Part 1

## Part 2

# Day 12

## Part 1

## Part 2

# Day 13

## Part 1

## Part 2

# Day 14

## Part 1

## Part 2

# Day 15

## Part 1

## Part 2

# Day 16

## Part 1

## Part 2

# Day 17

## Part 1

## Part 2

# Day 18

## Part 1

## Part 2

# Day 19

## Part 1

## Part 2

# Day 20

## Part 1

## Part 2

# Day 21

## Part 1

## Part 2

# Day 22

## Part 1

## Part 2

# Day 23

## Part 1

## Part 2

# Day 24

## Part 1

## Part 2

# Day 25

## Part 1

## Part 2